In [ ]:
import librosa
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import os
from PIL import Image
import pathlib
import csv
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
import keras
from keras import layers
from keras import layers
import keras
from keras.models import Sequential
import warnings
warnings.filterwarnings('ignore')

In [ ]:
!wget https://www.dropbox.com/s/4jw31k5mlzcmgis/genres.tar.gz

--2024-01-15 11:11:17--  https://www.dropbox.com/s/4jw31k5mlzcmgis/genres.tar.gz
Resolving www.dropbox.com (www.dropbox.com)... 162.125.3.18, 2620:100:6016:18::a27d:112
Connecting to www.dropbox.com (www.dropbox.com)|162.125.3.18|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: /s/raw/4jw31k5mlzcmgis/genres.tar.gz [following]
--2024-01-15 11:11:17--  https://www.dropbox.com/s/raw/4jw31k5mlzcmgis/genres.tar.gz
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uceba2a593c9f291352e1785dd31.dl.dropboxusercontent.com/cd/0/inline/CLY3o1WDXGwakSO4osRlT6IStKcbCEuGqoPyxwFcWJQGfOUQ1vs59RCYFfAVViEu39-jpS_qC2knmc61hVWDLMRZgHajG-nALKGfv5_AJsvfSijf5zYXnf5Eb5P-3oDTD08/file# [following]
--2024-01-15 11:11:18--  https://uceba2a593c9f291352e1785dd31.dl.dropboxusercontent.com/cd/0/inline/CLY3o1WDXGwakSO4osRlT6IStKcbCEuGqoPyxwFcWJQGfOUQ1vs59RCYFfAVViEu39-jpS_qC2knmc61hVWDLMRZgHajG-nALKGfv5_AJsvfSijf5zYXnf5E

In [ ]:

# importing the "tarfile" module
import tarfile

# open file
file = tarfile.open('genres.tar.gz')

# extracting file
file.extractall('./audio2')

file.close()

In [ ]:
#import os
#os.rename('genres.tar.gz?dl=0','audio1.zip')


In [ ]:
#from google.colab import drive
#drive.mount('/mnt/drive')

#for i in os.walk('/mnt/drive/MyDrive/audio/genres'):
 # print (i)
#dataset_path='/mnt/drive/MyDrive/audio/genres'
#print(dataset_path+'{g}')

In [ ]:
cmap = plt.get_cmap('inferno')
plt.figure(figsize=(8,8))
genres = 'blues classical country disco hiphop jazz metal pop reggae rock'.split()
for g in genres:
    pathlib.Path(f'img_data/{g}').mkdir(parents=True, exist_ok=True)
    for filename in os.listdir(f'./audio2/genres/{g}'):
        songname = f'./audio2/genres/{g}/{filename}'
        y, sr = librosa.load(songname, mono=True, duration=5)
        plt.specgram(y, NFFT=2048, Fs=2, Fc=0, noverlap=128, cmap=cmap, sides='default', mode='default', scale='dB');
        plt.axis('off');
        plt.savefig(f'img_data/{g}/{filename[:-3].replace(".", "")}.png')
        plt.clf()

<Figure size 800x800 with 0 Axes>

In [ ]:
header = 'filename chroma_stft rmse spectral_centroid spectral_bandwidth rolloff zero_crossing_rate'
for i in range(1, 21):
    header += f' mfcc{i}'
header += ' label'
header = header.split()

In [ ]:
file = open('dataset.csv', 'w', newline='')
with file:
    writer = csv.writer(file)
    writer.writerow(header)
genres = 'blues classical country disco hiphop jazz metal pop reggae rock'.split()
for g in genres:
    for filename in os.listdir(f'./audio2/genres/{g}'):
        songname = f'./audio2/genres/{g}/{filename}'
        y, sr = librosa.load(songname, mono=True, duration=30)
        rmse = librosa.feature.rms(y=y)
        chroma_stft = librosa.feature.chroma_stft(y=y, sr=sr)
        spec_cent = librosa.feature.spectral_centroid(y=y, sr=sr)
        spec_bw = librosa.feature.spectral_bandwidth(y=y, sr=sr)
        rolloff = librosa.feature.spectral_rolloff(y=y, sr=sr)
        zcr = librosa.feature.zero_crossing_rate(y)
        mfcc = librosa.feature.mfcc(y=y, sr=sr)
        to_append = f'{filename} {np.mean(chroma_stft)} {np.mean(rmse)} {np.mean(spec_cent)} {np.mean(spec_bw)} {np.mean(rolloff)} {np.mean(zcr)}'
        for e in mfcc:
            to_append += f' {np.mean(e)}'
        to_append += f' {g}'
        file = open('dataset.csv', 'a', newline='')
        with file:
            writer = csv.writer(file)
            writer.writerow(to_append.split())

In [ ]:
data = pd.read_csv('dataset.csv')
data.head()# Dropping unneccesary columns
data = data.drop(['filename'],axis=1)#Encoding the Labels
genre_list = data.iloc[:, -1]
encoder = LabelEncoder()
y = encoder.fit_transform(genre_list)#Scaling the Feature columns
scaler = StandardScaler()
X = scaler.fit_transform(np.array(data.iloc[:, :-1], dtype = float))#Dividing data into training and Testing set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [ ]:
model = Sequential()
model.add(layers.Dense(256, activation='relu', input_shape=(X_train.shape[1],)))
model.add(layers.Dense(128, activation='relu'))
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(10, activation='softmax'))
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
callback = keras.callbacks.EarlyStopping(monitor='loss')

In [ ]:
#classifier = model.fit(X_train, y_train,  epochs=100, batch_size=128)
model.fit(X_train, y_train,
          epochs=400,batch_size=128,
          verbose=1,
          validation_data=(X_test, y_test))

Epoch 1/400
7/7 [==============================] - 0s 50ms/step - loss: 3.4471e-06 - accuracy: 1.0000 - val_loss: 2.2716 - val_accuracy: 0.7550
Epoch 2/400
7/7 [==============================] - 0s 16ms/step - loss: 3.4328e-06 - accuracy: 1.0000 - val_loss: 2.2732 - val_accuracy: 0.7550
Epoch 3/400
7/7 [==============================] - 0s 17ms/step - loss: 3.4186e-06 - accuracy: 1.0000 - val_loss: 2.2713 - val_accuracy: 0.7550
Epoch 4/400
7/7 [==============================] - 0s 18ms/step - loss: 3.4052e-06 - accuracy: 1.0000 - val_loss: 2.2731 - val_accuracy: 0.7550
Epoch 5/400
7/7 [==============================] - 0s 14ms/step - loss: 3.3898e-06 - accuracy: 1.0000 - val_loss: 2.2730 - val_accuracy: 0.7550
Epoch 6/400
7/7 [==============================] - 0s 18ms/step - loss: 3.3806e-06 - accuracy: 1.0000 - val_loss: 2.2751 - val_accuracy: 0.7550
Epoch 7/400
7/7 [==============================] - 0s 17ms/step - loss: 3.3736e-06 - accuracy: 1.0000 - val_loss: 2.2780 - val_accuracy: